In [4]:
import netCDF4 as nc
import numpy as np
file = '/shared_space/G1/gedi/GEDI04_A_2021284160027_O16033_04_T11110_02_002_02_V002.h5'
fp = nc.Dataset(file)
print(fp)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    short_name: GEDI_L4A
    dimensions(sizes): 
    variables(dimensions): 
    groups: ANCILLARY, BEAM0000, BEAM0001, BEAM0010, BEAM0011, BEAM0101, BEAM0110, BEAM1000, BEAM1011, METADATA



In [8]:
import xarray as xr

file = '/shared_space/G1/gedi/GEDI04_A_2021284160027_O16033_04_T11110_02_002_02_V002.h5'
fp = xr.open_dataset(file, group='BEAM0000')
print(fp)

<xarray.Dataset>
Dimensions:               (phony_dim_4: 63842, phony_dim_5: 4)
Coordinates:
    delta_time            (phony_dim_4) float64 ...
    lat_lowestmode        (phony_dim_4) float64 ...
    lon_lowestmode        (phony_dim_4) float64 ...
Dimensions without coordinates: phony_dim_4, phony_dim_5
Data variables:
    agbd                  (phony_dim_4) float32 ...
    agbd_pi_lower         (phony_dim_4) float32 ...
    agbd_pi_upper         (phony_dim_4) float32 ...
    agbd_se               (phony_dim_4) float32 ...
    agbd_t                (phony_dim_4) float32 ...
    agbd_t_se             (phony_dim_4) float32 ...
    algorithm_run_flag    (phony_dim_4) uint8 ...
    beam                  (phony_dim_4) uint16 ...
    channel               (phony_dim_4) uint8 ...
    degrade_flag          (phony_dim_4) uint8 ...
    elev_lowestmode       (phony_dim_4) float32 ...
    l2_quality_flag       (phony_dim_4) uint8 ...
    l4_quality_flag       (phony_dim_4) uint8 ...
    master_fr

In [9]:
print(fp['agbd'])

<xarray.DataArray 'agbd' (phony_dim_4: 63842)>
array([-9999., -9999., -9999., ..., -9999., -9999., -9999.], dtype=float32)
Coordinates:
    delta_time      (phony_dim_4) float64 ...
    lat_lowestmode  (phony_dim_4) float64 ...
    lon_lowestmode  (phony_dim_4) float64 ...
Dimensions without coordinates: phony_dim_4
Attributes:
    long_name:    Aboveground biomass density
    units:        Mg / ha
    description:  Aboveground biomass density (Mg / ha)


In [20]:
(fp['agbd'])

<xarray.DataArray 'agbd' (phony_dim_4: 63842)>
array([-9999., -9999., -9999., ..., -9999., -9999., -9999.], dtype=float32)
Coordinates:
    delta_time      (phony_dim_4) float64 1.192e+08 1.192e+08 ... 1.192e+08
    lat_lowestmode  (phony_dim_4) float64 0.4242 0.4237 0.4233 ... -25.69 -25.69
    lon_lowestmode  (phony_dim_4) float64 28.02 28.02 28.02 ... 48.38 48.38
Dimensions without coordinates: phony_dim_4
Attributes:
    long_name:    Aboveground biomass density
    units:        Mg / ha
    description:  Aboveground biomass density (Mg / ha)

In [25]:
bbox = [29.269398, -1.554519, 29.482104, -1.375895]
bbox[0]

29.269398

In [60]:
%%bash 

gdalwarp -overwrite -te 29.269398 -1.554519 29.482104 -1.375895 /shared_space/G1/gedi/GEDI04_A_2021284160027_O16033_04_T11110_02_002_02_V002.h5 test.h5
#Use gdalinfo to see what it created
#gdalinfo test.h5

ERROR 1: Cannot guess driver for test.h5


CalledProcessError: Command 'b'\ngdalwarp -overwrite -te 29.269398 -1.554519 29.482104 -1.375895 /shared_space/G1/gedi/GEDI04_A_2021284160027_O16033_04_T11110_02_002_02_V002.h5 test.h5\n#Use gdalinfo to see what it created\n#gdalinfo test.h5\n'' returned non-zero exit status 1.

In [50]:
%%bash
gdalinfo /shared_space/G1/gedi/GEDI04_A_2021284160027_O16033_04_T11110_02_002_02_V002.h5

Driver: HDF5/Hierarchical Data Format Release 5
Files: /shared_space/G1/gedi/GEDI04_A_2021284160027_O16033_04_T11110_02_002_02_V002.h5
Size is 512, 512
Coordinate System is `'
Metadata:
  BEAM0000_agbd_coordinates=delta_time lat_lowestmode lon_lowestmode
  BEAM0000_agbd_description=Aboveground biomass density (Mg / ha)
  BEAM0000_agbd_long_name=Aboveground biomass density
  BEAM0000_agbd_pi_lower_coordinates=delta_time lat_lowestmode lon_lowestmode
  BEAM0000_agbd_pi_lower_description=Lower prediction interval (see alpha attribute for the level)
  BEAM0000_agbd_pi_lower_long_name=Aboveground biomass density lower prediction interval
  BEAM0000_agbd_pi_lower_units=Mg / ha
  BEAM0000_agbd_pi_upper_coordinates=delta_time lat_lowestmode lon_lowestmode
  BEAM0000_agbd_pi_upper_description=Upper prediction interval (see alpha attribute for the level)
  BEAM0000_agbd_pi_upper_long_name=Aboveground biomass density upper prediction interval
  BEAM0000_agbd_pi_upper_units=Mg / ha
  BEAM0000_agbd

In [55]:
%%bash

gdalinfo HDF5:"/shared_space/G1/gedi/GEDI04_A_2021284160027_O16033_04_T11110_02_002_02_V002.h5"://BEAM0000/agbd_prediction/xvar_a1

Driver: HDF5Image/HDF5 Dataset
Files: /shared_space/G1/gedi/GEDI04_A_2021284160027_O16033_04_T11110_02_002_02_V002.h5
Size is 4, 63842
Coordinate System is `'
Metadata:
  BEAM0000_agbd_prediction_alpha=0.1 
  BEAM0000_agbd_prediction_l2a_alg_count=7 
  BEAM0000_agbd_prediction_max_nvar=4 
  BEAM0000_agbd_prediction_pft_grid_version=4 
  BEAM0000_agbd_prediction_pft_infilled_grid_version=6 
  BEAM0000_agbd_prediction_phenology_grid_version=5 
  BEAM0000_agbd_prediction_predictor_offset=100 
  BEAM0000_agbd_prediction_region_grid_version=5 
  BEAM0000_agbd_prediction_response_offset=0 
  BEAM0000_agbd_prediction_urban_grid_version=1 
  BEAM0000_agbd_prediction_water_grid_version=1 
  BEAM0000_description=Coverage beam
  BEAM0001_agbd_prediction_alpha=0.1 
  BEAM0001_agbd_prediction_l2a_alg_count=7 
  BEAM0001_agbd_prediction_max_nvar=4 
  BEAM0001_agbd_prediction_pft_grid_version=4 
  BEAM0001_agbd_prediction_pft_infilled_grid_version=6 
  BEAM0001_agbd_prediction_phenology_grid_version=

HDF5-DIAG: Error detected in HDF5 (1.10.4) thread 140118713964608:
  #000: H5Adeprec.c line 202 in H5Aopen_name(): can't open attribute: '_FillValue'
    major: Attribute
    minor: Can't open object
  #001: H5Aint.c line 602 in H5A__open_by_name(): unable to load attribute info from object header
    major: Attribute
    minor: Unable to initialize object
  #002: H5Oattribute.c line 515 in H5O__attr_open_by_name(): can't locate attribute: '_FillValue'
    major: Attribute
    minor: Object not found


In [56]:
%%bash
gdal_translate -of GTiff HDF5:"/shared_space/G1/gedi/GEDI04_A_2021284160027_O16033_04_T11110_02_002_02_V002.h5"://BEAM0000/agbd_prediction/xvar_a1 output.tif

Input file size is 4, 63842
0...10...20...30...40...50...60...70...80...90...100 - done.


HDF5-DIAG: Error detected in HDF5 (1.10.4) thread 140678922884544:
  #000: H5Adeprec.c line 202 in H5Aopen_name(): can't open attribute: '_FillValue'
    major: Attribute
    minor: Can't open object
  #001: H5Aint.c line 602 in H5A__open_by_name(): unable to load attribute info from object header
    major: Attribute
    minor: Unable to initialize object
  #002: H5Oattribute.c line 515 in H5O__attr_open_by_name(): can't locate attribute: '_FillValue'
    major: Attribute
    minor: Object not found
